In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
from dataclasses import dataclass, field
from PIL import Image
import torch
from torch import nn
import torch.nn.functional as F
import os
import json
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from datetime import datetime

from typing import List

import requests
from pysolotools.consumers import Solo
from pysolotools.converters.solo2coco import SOLO2COCOConverter
from pysolotools.core.models import KeypointAnnotationDefinition, RGBCameraCapture
from typing import Dict, List, Tuple
from pysolotools.core.models import Frame, BoundingBox2DLabel, BoundingBox2DAnnotation

import torch
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
from torchvision.models import resnet18, ResNet18_Weights

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

import base_new as base

np.set_printoptions(suppress=True)
# %matplotlib ipympl

c:\Users\Leonard\miniconda3\envs\openmmlab\lib\site-packages\mmcv\__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [3]:
weights = ResNet18_Weights.IMAGENET1K_V1
resnet = resnet18(weights=weights)
resnet = torch.nn.Sequential(*list(resnet.children())[:-1])
resnet = resnet.eval().cuda()
preprocess = weights.transforms()

config_file = '../../Projects/mmdetection/configs/swin/mask_rcnn_swin-t-p4-w7_fpn_fp16_ms-crop-3x_coco.py'
checkpoint_file = '../../Projects/mmdetection/checkpoints/mask_rcnn_swin-t-p4-w7_fpn_fp16_ms-crop-3x_coco_20210908_165006-90a4008c.pth'
obj_det = init_detector(config_file, checkpoint_file, device='cuda:0')

load checkpoint from local path: ../../Projects/mmdetection/checkpoints/mask_rcnn_swin-t-p4-w7_fpn_fp16_ms-crop-3x_coco_20210908_165006-90a4008c.pth


In [4]:
folder_dir = 'C:/Users/Leonard/AppData/LocalLow/DefaultCompany/Perception2/solo_16'
solo = Solo(data_path=folder_dir)

Each frame has multi-capture instances from multi-camera  
Each capture has multiple annotations, one of which is a 2DBoundingBox annotation  
For the 2DBoundingBox annotation, there is a list of 2DBoundingBoxLabel represents the number of objects  
We convert the 2DBoundingBoxLabel to Custom2DBoundingBoxLabel to hold the box's light ray  

In [34]:
reload(base)

pred_errs = []
for frame_idx, frame in enumerate(solo.frames()):
    # print(f'\r{frame_idx}/{len(solo.frames())}', end='')
    deep_frame = base.DeepFrame.from_frame(frame, f'{solo.data_path}/sequence.{frame_idx}', obj_det, resnet, preprocess)
    for instanceId, item in deep_frame.loc_gt.items():
        # print(instanceId, item['pred_err'])
        pred_errs.append(item['pred_err'])
    break

print()
print(np.mean(pred_errs), np.var(pred_errs), np.min(pred_errs), np.max(pred_errs))

LocationLabel(metadata={}, instanceId=1, labelId=1, position=array([3.529252  , 1.11557269, 1.5095558 ]))
LocationLabel(metadata={}, instanceId=2, labelId=1, position=array([1.47503769, 1.11557269, 1.8960123 ]))
step0.camera_0.png
LocationLabel(metadata={}, instanceId=1, labelId=1, position=array([3.5292511 , 1.11557269, 1.5095576 ]))
LocationLabel(metadata={}, instanceId=2, labelId=1, position=array([1.4750366 , 1.11557269, 1.89601445]))
step0.camera.png

0.02257075310730816


In [25]:
np.mean(pred_errs)

0.03129284307616376

In [6]:
pred = np.array([3.51531737, 1.10592773, 1.50455604])
gt = np.array([3.529252, 1.11557269, 1.5095558])

print(np.sqrt((pred - gt)**2).sum())

0.028579350000000003
